# Packages

In [1]:
from os import getcwd as gwd
from os import chdir as cd
from itertools import combinations 
from scipy.stats import pearsonr as pr
from scipy.stats import zscore as zscore
from scipy.stats import ttest_rel as ttest
import copy
import numpy as np
import pandas
import matplotlib.pyplot as plt
np.random.seed(223)

# Data

In [122]:
subjs = ['AN','AN2','AO','EW','IB','KH','MH','MK','MY','NP']
dataAll = dict()
indcounter = 1
for subjName in subjs:
    data = pandas.read_csv((subjName+'.csv'),header=None,names=['cond','x','y','loc'],usecols=[13, 26, 27, 35])
    data = data[-pandas.isnull(data.x)] #delete the initial lines
    data = data[1:769] #take the rows we want
    data.index = range(768)
    data.cond[data.cond == 'womanmask.png'] = 1
    data.cond[data.cond == 'noisePatch.png'] = 2
    data = data.astype(float)
    condarray = np.empty(768, dtype = np.int)
    condarray.fill(indcounter)
    data['ind'] = condarray
    indcounter += 1
    dataAll[subjName] = data

In [123]:
dataAll['EW']

,cond,x,y,loc,ind
0,1.0,-0.170573,-0.326823,81.0,4
1,2.0,-0.281250,0.285156,18.0,4
2,1.0,-0.410156,-0.110677,74.0,4
3,1.0,-0.368490,-0.029948,73.0,4
4,2.0,0.092448,0.317708,10.0,4
...,...,...,...,...,...
763,1.0,0.186198,-0.326823,88.0,4
764,1.0,0.335938,-0.089844,94.0,4
765,1.0,-0.160156,0.260417,64.0,4
766,1.0,-0.322917,-0.145833,75.0,4


# Functions

In [129]:
#Calculate angular response
def calAng(x, y):
    angResp = np.rad2deg(np.arctan2(y,x))
    if angResp < 0:
        angResp += 360
    return angResp

#Morph errors---what exactly does this do???
def morphErr(err):
    if err > 180:
        err = -360+err
    else:
        if err < (-180):
            err = 360+err
    return err

#Replace outliers with mean
def clearOutliers(data):
    avg = np.mean(data, axis = None)
    sd = np.mean(data, axis = None)
    out1 = np.nonezero(data < float(avg - 3*sd))
    out2 = np.nonezero(data > float(avg + 3*sd))
    data[out1], data[out2] = avg
    return data

def cuthalf(x):
    if x > 383:
        x -= 383
    return x

# Preprocessing

In [130]:
def prepersubj(data):
    angerr = None
    for trials in range(0, 768):
        err = morphErr(calAng(data.x[trials], data.y[trials]) - data.loc[cuthalf(trials)]*7.5)
        if angerr is not None:
            angerr = np.vstack((angerr, np.array(err)))
        else:
            angerr = np.array(err)
    angerr = clearoutlier(angerr)
    return angerr

In [131]:
errAll = dict()
for subj in subjs:   
    errs = pandas.DataFrame(data = prepersubj(dataAll[subj]),columns=['errors'])
    errs['locs'] = dataAll[subj].loc
    errs['cond'] = dataAll[subj].cond
    errs['ind'] = dataAll[subj].ind
    errs = errs.sort_values('locs')
    errs.index = range(768)
    errAll[subj] = errs

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().